# Create a Pharos class input file

This notebook creates a file that contains gene symbols, Ensembl IDs, UniProt IDs, and Pharos class for each gene. The steps are:
1. Query Pharos for all targets in their database
2. Parse the JSON response as a data frame
3. Merge this data with Agora's existing UniProt -> Ensembl ID map file

Step 3 simultaneously maps the IDs given by Pharos (UniProt IDs and gene symbols only) to the corresponding Ensembl IDs, and narrows the data down to only genes that exist in Agora's data files.

In [ ]:
import requests
import json
import pandas as pd
import synapseclient

## Query Pharos for all targets in the database

Pharos uses GraphQL for its API, so the query is formatted as JSON that matches their "DownloadResult" schema. The lone "data" value inside signifies to return the data only, not any status or metadata about the request.

In [ ]:
url = "https://pharos-api.ncats.io/graphql"

query_body = """
{
    download(model: "Targets", fields: ["UniProt", "Symbol", "Target Development Level"], sqlOnly: false) {
        data
    }
}
"""

response = requests.post(url=url, json={"query": query_body})

if not response.ok:
    print("Error querying Pharos")
    response.raise_for_status()

## Turn the response into a DataFrame

The response is a JSON string with the following structure:

```
data {
    download {
        data {
            [list of dictionary items with fields "id", "UniProt", "Symbol", and "Target Development Level"]]
        }
    }
}
```

Calling `json_normalize` on the inner "data" item will turn that list into a data frame. 

In [ ]:
res_str = json.loads(response.content)
pharos_df = pd.json_normalize(res_str["data"]["download"]["data"])

pharos_df = pharos_df.rename(
    columns={
        "UniProt": "uniprot_id",
        "Symbol": "hgnc_symbol",
        "Target Development Level": "pharos_class",
    }
).drop(columns="id")

pharos_df

## Map UniProt IDs to Ensembl IDs

Uses the UniProt -> Ensembl ID Agora source file

In [ ]:
# Temporarily hard-coded until this file is in the config.yaml file
uniprot_syn_id = "syn54113663"

syn = synapseclient.Synapse()
syn.login(silent=True)

uniprot_path = syn.get(uniprot_syn_id)
uniprot_df = pd.read_table(uniprot_path.path, sep="\t")

uniprot_df

In [ ]:
final_df = pd.merge(
    left=pharos_df,
    right=uniprot_df,
    how="inner",
    left_on="uniprot_id",
    right_on="UniProtKB_accession",
    validate="one_to_many",
).rename(columns={"RESOURCE_IDENTIFIER": "ensembl_gene_id"})

final_df = final_df[
    ["ensembl_gene_id", "uniprot_id", "hgnc_symbol", "pharos_class"]
].sort_values(by="ensembl_gene_id")

final_df

## Save to a file

In [ ]:
final_df.to_csv("../../output/pharos_classes.csv", index=False)